In [1]:
### no brand names and search terms vector

In [1]:
%load_ext Cython

In [2]:
%%time
%%cython
from gensim.models import doc2vec
import nltk
import re
def split_sentence (sentence):
    return re.split('\W+',sentence)
    
class MyDocs(object):
    def __iter__(self):
        for i, text in enumerate(open("txt_pt.txt")): # doesn't use brand name
            yield doc2vec.LabeledSentence(words=split_sentence(text), tags=['%s' % i])
            # Train the doc2vec model
cdef desp = MyDocs()
model = doc2vec.Doc2Vec(desp, size = 200, window = 8, min_count = 5, workers = 4)
model.save('distance_pt_nst.model')

CPU times: user 1min 25s, sys: 16.5 s, total: 1min 42s
Wall time: 1min 2s


In [3]:
model = doc2vec.Doc2Vec.load('distance_pt_nst.model')

In [4]:
import numpy as np
st_arrays = np.zeros((240760, 200))
for i,text in enumerate(open("txt_st.txt")): # input search terms 
    st_arrays[i]=model.infer_vector(split_sentence(text))   
st_arrays

array([[ 0.00880427, -0.05650265, -0.0568384 , ..., -0.03371122,
         0.00039218,  0.11478763],
       [-0.0412861 , -0.05030178, -0.04477246, ..., -0.03367533,
         0.0354549 , -0.01867473],
       [ 0.03970589, -0.04723416, -0.05234999, ..., -0.02919966,
         0.08714339,  0.0619088 ],
       ..., 
       [-0.01021531, -0.02577889, -0.02534036, ..., -0.07705922,
         0.07093358,  0.05344896],
       [-0.01204387, -0.02524005, -0.02596355, ..., -0.07782666,
         0.07232198,  0.05347389],
       [ 0.01465103, -0.01340673, -0.02702369, ..., -0.09780332,
         0.04759386,  0.02608644]])

In [5]:
## calculate the cosin distance between 2 vectors to generate the 3rd features
# Compute the cosine similarity values between the input text and all archived reviews
# cossims_with_input = map(lambda v: cossim(input_vec, v), model.docvecs) 
# need to chaneg the code into rows: calculate the cosin distance between the vectors in same rows
import numpy as np
import pandas as pd
# Calculate the cosine similarity between two vecotrs 
def cossim(v1, v2):
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2))

cossims_with_st=np.zeros((240760,1))
for i in range(240760):
    cossims_with_st[i] = cossim(st_arrays[i], model.docvecs[i])

In [6]:
np.savetxt("distance_pt_nst.csv",cossims_with_st, delimiter=",")

In [7]:
coswithst=pd.read_csv("distance_pt_nst.csv",header=None)
coswithst


,0
0,0.187009
1,0.139878
2,0.057679
3,-0.176724
4,-0.254143
5,0.125256
6,0.203047
7,0.047217
8,0.023340
9,0.288531


In [ ]:
cws_trn=coswithst.iloc[:num_train]
cws_tst=coswithst.iloc[num_train:]